<a href="https://colab.research.google.com/github/marco-siino/gpt_energy_cost_optimization/blob/main/GPT_EnergyCostOptimization_evaluator_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the libraries required.

In [37]:
import requests
import zipfile
import io
import calendar
import re
from datetime import datetime

# Get the log files from GitHub and extract the folders.

In [4]:
def get_log_files_zipped (zip_url, folder_name):
  # Send a GET request to the URL
  response = requests.get(zip_url)

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Read the content of the response
      zip_content = io.BytesIO(response.content)

      # Extract the zip file
      with zipfile.ZipFile(zip_content, 'r') as zip_ref:
          zip_ref.extractall('/content/'+folder_name)  # Extract to the '/content/' directory in Colab
      print("Zip file extracted successfully.")
  else:
      print("Failed to download zip file.")

In [6]:
# GitHub repository URL
jul2014_url = 'https://github.com/marco-siino/gpt_energy_cost_optimization/raw/main/multiple_appliances_jul2014.zip'
nov2014_url = 'https://github.com/marco-siino/gpt_energy_cost_optimization/raw/main/multiple_appliances_nov2014.zip'
feb2015_url = 'https://github.com/marco-siino/gpt_energy_cost_optimization/raw/main/multiple_appliances_feb2015.zip'
apr2015_url = 'https://github.com/marco-siino/gpt_energy_cost_optimization/raw/main/multiple_appliances_apr2015.zip'

get_log_files_zipped(jul2014_url,"jul2014")
get_log_files_zipped(nov2014_url,"nov2014")
get_log_files_zipped(feb2015_url,"feb2015")
get_log_files_zipped(apr2015_url,"apr2015")



Zip file extracted successfully.
Zip file extracted successfully.
Zip file extracted successfully.
Zip file extracted successfully.


# Function to get the content of a log file.

In [25]:
def get_log_file_content(day,month,year):
  month_number = int(month)
  month_name = calendar.month_name[month_number]
  # Get the first three characters of the month name
  short_month_name = month_name[:3].lower()  # Convert to lowercase for consistency
  file_path = short_month_name+str(year)+"/multiple_appliances/multiple_appliances_"+str(year)+"-"+str(month)+"-"+str(day)+".txt"
  file_content=""
  print(file_path)
  try:
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Read the contents of the file into a string
        file_content = file.read()
        print("File content stored in string successfully.")
  except FileNotFoundError:
    print("File not found. Please provide the correct file path.")
  except Exception as e:
    print("An error occurred:", e)
  return file_content

# Given a daily log file, function to count and return the number of actual intervals and the number of Mistral-proposed intervals.

In [30]:
def calculate_missing_intervals(day,month,year):
  file_content = get_log_file_content(day,month,year)
  split_string = file_content.split("Mistral operating times:", 1)

  actual_intervals = split_string[0]
  mistral_intervals = split_string[1]

  nr_actual_intervals = actual_intervals.count("From")
  nr_mistral_intervals = mistral_intervals.count("from")

  return nr_actual_intervals,nr_mistral_intervals

# Given a daily log file, function to parse the Mistral proposal and to evaluate if a given interval is acceptable or not.
The function returns the total number of proposed intervals and the number of acceptable intervals.

In [39]:
def calculate_acceptable_intervals(day,month,year):
  file_content = get_log_file_content(day,month,year)
  split_string = file_content.split("Mistral operating times:", 1)

  mistral_intervals = split_string[1]

  lines = mistral_intervals.split('\n')

  nr_acceptable_intervals=0
  for line in lines:
        # Define the regular expression pattern to match words in the format xx:xx:xx
        pattern = r'\b\d{2}:\d{2}:\d{2}\b'

        # Find all occurrences of the pattern in the input string
        matches = re.findall(pattern, line)

        # Extract the first and second occurrences
        from_time = matches[0] if matches else None
        to_time = matches[1] if len(matches) > 1 else None

        try:
          if (from_time != None)  and (to_time!=None):
            if from_time < to_time:
              nr_acceptable_intervals+=1
        except:
          continue

  return nr_acceptable_intervals

# Function to compute the evaluation metrics given a month.

In [54]:
def compute_evaluation_metrics(month,year):
  nr_total_actual_intervals = 0
  nr_total_mistral_proposed_intervals = 0
  nr_total_acceptable_intervals = 0
  last_day = 31

  # February 2015
  if month=="02" and year=="2015":
    last_day = 28
  if month=="04":
    last_day = 30
  if month=="07":
    last_day = 31
  if month=="11":
    last_day = 30

  for day in range(1,last_day+1):
    two_digits_day = "{:02d}".format(day)
    current_actual_intervals,current_mistral_proposed_intervals = calculate_missing_intervals(two_digits_day,month,year)
    nr_total_actual_intervals += current_actual_intervals
    nr_total_mistral_proposed_intervals += current_mistral_proposed_intervals
    nr_total_acceptable_intervals+=calculate_acceptable_intervals(two_digits_day,month,year)

  missing_intervals_ratio = 1-(nr_total_mistral_proposed_intervals/nr_total_actual_intervals)
  accuracy = nr_total_acceptable_intervals/nr_total_mistral_proposed_intervals

  print("The accuracy for "+month+ " is: "+str(accuracy))
  print("The missing intervals ratio for "+month+ " is: "+str(missing_intervals_ratio))

In [55]:
compute_evaluation_metrics("02","2015")

feb2015/multiple_appliances/multiple_appliances_2015-02-01.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-01.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-02.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-02.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-03.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-03.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-04.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-04.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_appliances_2015-02-05.txt
File content stored in string successfully.
feb2015/multiple_appliances/multiple_